In [ ]:
print('Hello LangChain')

In [14]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 


load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# print(OPENAI_API_KEY)

In [ ]:
# prompt
prompt = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 개발자입니다.") , 
     ("user", "{input}") ]
)
print(prompt)

prompt_text = prompt.format(input="LangServe는 무엇인가요? 자세하게 설명해주세요")

In [ ]:
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    # api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
print(llm)

In [ ]:
try:
    response = llm.invoke(prompt_text)
    print(type(response))
    # print(type(response.content))
    print("응답:", response.content)
except Exception as e:
    print(f"오류 발생: {e}")

### LCEL
- Prompt + LLM을 Chain으로 연결하기

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
    You are an expert in AI Expert. Answer the question. 
    <Question>: {input}에 대해 쉽게 반드시 한글로 설명해주세요.")
    """)                                     
prompt

In [ ]:
# chain 연결 (LCEL)
chain = prompt | llm
print(type(chain))

In [ ]:
from langchain_core.output_parsers import StrOutputParser

# chain 연결 (LCEL)
output_parser = StrOutputParser()

chain2 = prompt | llm | output_parser
print(type(chain2))

### LCEL
- Prompt + LLM + OutputParser을 Chain으로 연결하기

In [ ]:
from langchain_core.output_parsers import StrOutputParser

# chain 연결 (LCEL) prompt + llm + outputparser
output_parser = StrOutputParser()

chain2 = prompt | llm | output_parser
print(type(chain2))

In [ ]:
# chain 호출
try:
    result = chain.invoke({"input": "인공지능 모델의 학습 원리"})
    print(type(result))
    print(result.content)
except Exception as e:
    print(f"오류 발생: {e}")

In [ ]:
# chain 호출
try:
    result = chain2.invoke({"input": ": LangChain의 Products(제품)는 어떤 것들이 있나요?"})
    print(type(result))
    print(result)
except Exception as e:
    print(f"오류 발생: {e}")

### Runnable의 stream() 함수 호출

In [ ]:
# 스트리밍 출력을 위한 요청
try: 
    answer = chain2.stream({"input": "인공지능 모델의 학습 원리를 자세하게 설명해 주세요."})

    # 스트리밍 출력
    # print(answer)
    for token in answer:
        print(token, end="", flush=True)
except Exception as e: 
    print(f"오류 발생: {e}")

### Multi Chain
- 첫번째 Chain의 출력이, 두번째 Chain의 입력이 된다.
- 두 개의 Chain과 Prompt + OutputParser를 LCEL로 연결하기

In [26]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Step 1: 사용자가 입력한 장르에 따라 영화 추천
prompt1 = ChatPromptTemplate.from_template("{genre} 장르에서 추천할 만한 영화를 한 편 알려주세요.")

# Step 2: 추천된 영화의 줄거리를 요약
prompt2 = ChatPromptTemplate.from_template("{movie} 추전한 영화의 제목을 먼저 알려주시고, 줄을 바꾸어서 영화의 줄거리를 3문장으로 요약해 주세요.")

# OpenAI 모델 사용
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    #model="mistral-saba-24b",
    temperature=0.7
)

# 체인 1: 영화 추천 (입력: 장르 → 출력: 영화 제목)
chain1 = prompt1 | llm | StrOutputParser()

In [ ]:
# 체인 2: 줄거리 요약 (입력: 영화 제목 → 출력: 줄거리)
chain2 = (
    {"movie": chain1}  # chain1의 출력을 movie 변수로 전달
    | prompt2
    | llm
    | StrOutputParser()
)

# 실행: "SF" 장르의 영화 추천 및 줄거리 요약
response = chain2.invoke({"genre": "액션"})
print(response)  

### PromptTemplate 여러개 연결하기

In [ ]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 요약해서 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(model_name="ChatGPT", count=3)

# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요")
              + "\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요."
)
combined_prompt.format(model_name="ChatGPT", count=3, language="영어")

# OpenAI 모델 사용
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

chain = combined_prompt | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3, "language":"영어"})
print(response)

In [ ]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 요약해서 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

questions = [
    {"model_name": "GPT-4", "count": 2},
    {"model_name": "Gemma", "count": 3},
    {"model_name": "llama-4", "count": 4},
]

# 여러 개의 프롬프트를 미리 생성
formatted_prompts = [prompt_template.format(**q) for q in questions]
print(formatted_prompts)  # 미리 생성된 질문 목록 확인

In [ ]:
# OpenAI 모델 사용
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

for prompt in formatted_prompts:
    response = llm.invoke(prompt) #AIMessage
    print(response.content)

### SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate 사용한 ChatPromptTemplate

In [ ]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate
)
from langchain_openai import ChatOpenAI

# 개별 메시지 템플릿 정의
system_message = SystemMessagePromptTemplate.from_template(
    "당신은 {topic} 전문가입니다. 명확하고 자세하게 설명해 주세요."
)
user_message = HumanMessagePromptTemplate.from_template(
    "{question}"
)
ai_message = AIMessagePromptTemplate.from_template(
    "This is an example answer about {topic}."
)

# ChatPromptTemplate로 메시지들을 묶기
chat_prompt = ChatPromptTemplate.from_messages([
    system_message,
    user_message,
    ai_message
])

# 메시지 생성
messages = chat_prompt.format_messages(topic="AI", question="딥러닝은 무엇인가요?")

# LLM 호출
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
response = llm.invoke(messages)

# 결과 출력
print(response.content)

### FewShotPromptTemplate
- 예시를 제공 프롬프트

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI

examples = [
    {
        "input": "뉴턴의 운동 법칙을 요약해 주세요.",
        "output": """### 뉴턴의 운동 법칙
1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.
2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.
3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다."""
    },
    {
        "input": "지구의 대기 구성 요소를 알려주세요.",
        "output": """### 지구 대기의 구성
- **질소 (78%)**: 대기의 대부분을 차지합니다.
- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.
- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.
- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다."""
    }
]

# 예제 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 초등학생도 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델 생성 및 체인 구성
#model = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
chain = final_prompt | model

# 테스트 실행
result = chain.invoke({"input": "태양계의 행성들을 간략히 정리해 주세요."})
#result = chain.invoke({"input": "양자 얽힘이 무엇인가요?"})
print(result.content)

### PartialPromptTemplate
- 프롬프트의 입력 값에 함수 호출 이나 외부 API를 호출한 동적인 값을 대입할 수 있음

In [ ]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month
    
    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# 프롬프트 템플릿 정의 (부분 변수 적용)

# prompt = PromptTemplate(
#     template="{season}에 일어나는 대표적인 지구과학 현상은 {phenomenon}입니다.",
#     input_variables=["phenomenon"],  # 사용자 입력 필요
#     partial_variables={"season": get_current_season("south")}  # 동적으로 계절 값 할당
# )

season = get_current_season("south")
print(f"현재 계절: {season}")

prompt = ChatPromptTemplate.from_template(
    "{season}에 주로 발생하는 대표적인 지구과학 현상 3가지를 알려주세요. "
    "각 현상에 대해 간단한 설명을 포함해주세요."
)

# OpenAI 모델 초기화
#model = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.5
)

# 특정 계절의 현상 질의
chain = (
    {"season": lambda x: season}
    | prompt
    | model
    | StrOutputParser()
)
result = chain.invoke({})


# 결과 출력
print(f"🔹 프롬프트: {prompt}")
print(f"🔹 모델 응답: {season}에 발생하는 자연 현상 :\n {result}")

In [ ]:
mport requests
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 실시간 환율을 가져오는 함수
def get_exchange_rate():
    response = requests.get("https://api.exchangerate-api.com/v4/latest/USD")
    data = response.json()
    return f"1달러 = {data['rates']['KRW']}원"

# {info} 변수에 API에서 받은 환율 정보를 동적으로 반영

prompt = PromptTemplate(
    template="현재 {info} 기준으로 환율 정보를 알려드립니다. 현재 환율을 기준으로 한국경제 미치는 영향 및 향후에 환율의 예상값에 대한 분석을 제공해 주세요.",
    input_variables=[],  # 사용자 입력 없음
    partial_variables={"info": get_exchange_rate()}  # API에서 가져온 데이터 자동 반영
)
print(prompt)

In [ ]:
# LLM 모델 설정 (GPT-4o-mini 사용)
#model = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.5
)

# 모델에 프롬프트 전달 및 응답 받기
response = model.invoke(prompt.format())

# 결과 출력
print("🔹 프롬프트:", prompt.format())
print("🔹 모델 응답:", response.content)